<a href="https://colab.research.google.com/github/mtjon/infompr-group-project/blob/feature%2Frnn/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title Generation through Abstract Summarisation with RNNs



In [1]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text==2.9.* tensorflow==2.9.*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 19 not upgraded.
Need to get 430 MB of archives.
After this operation, 1,392 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 13s (34.4 MB/s)
(Reading database ... 124016 files and directories currently installed.)
Removing libcudnn8-dev (8.1.1.33-1+cuda11.2) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.1.1.33-1+cuda11.2 t

In [2]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text
import tensorflow_text as tf_text

In [3]:
import requests

In [4]:
# get data
target_url = "https://raw.githubusercontent.com/EagleW/ACL_titles_abstracts_dataset/master/acl_titles_and_abstracts.txt"
response = requests.get(target_url)
raw_data = response.text
print(len(raw_data))

with open('acl_titles_and_abstract.txt', 'w') as file:
    file.writelines(raw_data)


8678084


In [5]:
import random
from random import shuffle
# Seed voor herhaalbaarheid
random.seed(42)

In [6]:
lines = []

# Code van https://github.com/EagleW/Writing-editing-Network/blob/master/split_data.py
with open("acl_titles_and_abstract.txt", 'r') as file1:
  lines=file1.readlines()

abs_t = []
abstracts = []
titles = []
i = 0
# TODO: possibly generates wrong tibs/abs when encountering a blank line; remove all blank lines
for line in lines:
    if i % 3 == 0:
        titles.append(line)
    elif i % 3 == 1:
        abstracts.append(line)
    i += 1
for i in range(len(abstracts)):
    if len(titles[i]) > 0 and len(abstracts[i]) > 0:
        h_a_pair = (titles[i], abstracts[i])
        abs_t.append(h_a_pair)
shuffle(abs_t)
total = len(abs_t)
dev = total//10
train  = total - dev - dev
i = 0

with open("val.txt", 'w') as valFile:
  for i in range(dev):
      valFile.writelines(abs_t[i][0])
      valFile.writelines(abs_t[i][1])
  #    valFile.writelines("\n")

with open("test.txt", 'w') as testFile:
  for i in range(dev, 2 * dev):
      testFile.writelines(abs_t[i][0])
      testFile.writelines(abs_t[i][1])
  #    file1.writelines("\n")

with open("train.txt", 'w') as trainFile:
  for i in range(2 * dev, total):
      trainFile.writelines(abs_t[i][0])
      trainFile.writelines(abs_t[i][1])
  #    trainFile.writelines("\n")


In [7]:
def get_abs_and_titles_from_raw(path_to_file):
    abstracts, titles = [], []
    with open(path_to_file) as data:
        lines = data.readlines()
        for abs in lines[1::2]:
            abstracts.append(abs.strip())
        for title in lines[0::2]:
            # // TODO: check if we need to add start and end tokens
            titles.append(title.strip())
            # titles.append('[START] ' + title.strip() + ' [END]')
    return abstracts, titles

In [8]:
train_text, train_labels = get_abs_and_titles_from_raw('train.txt')

train_dataset = tf.data.Dataset.from_tensor_slices((train_text, train_labels))
# ((train_text, train_inputs), train_labels) <-- tokenized

In [9]:
for abs, tit in train_dataset.batch(3).take(1):
    print('> Abstract examples:')
    for i, a in enumerate(abs.numpy()):
        print(i+1, a.decode('utf-8'))
        
    print()
    
    print('> Title examples:')
    for i, t in enumerate(tit.numpy()):
        print(i+1, t.decode('utf-8'))


> Abstract examples:
1 in this paper , we describe our participation in the tempeval-3 challenge . with our multilingual temporal tagger heideltime , we addressed task a , the extraction and normalization of temporal expressions for english and spanish . exploiting heideltimes strict separation between source code and languagedependent parts , we tuned heideltimes existing english resources and developed new spanish resources . for both languages , we achieved the best results among all participants for task a , the combination of extraction and normalization . both the improved english and the new spanish resources are publicly available with heideltime .
2 we present three novel methods of compactly storing very large n-gram language models . these methods use substantially less space than all known approaches and allow n-gram probabilities or counts to be retrieved in constant time , at speeds comparable to modern language modeling toolkits . our basic approach generates an explicit

In [10]:
# https://www.tensorflow.org/tutorials/load_data/text#example_1_predict_the_tag_for_a_stack_overflow_question
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250

# int_vectorize_layer = tf.keras.layers.TextVectorization(
#     max_tokens=VOCAB_SIZE,
#     output_mode='int',
#     output_sequence_length=MAX_SEQUENCE_LENGTH
#     )

int_vectorize_layer = tf.keras.layers.TextVectorization(
    output_mode='int'
    )

int_vectorize_layer.adapt(train_dataset)
print(len(int_vectorize_layer.get_vocabulary()))


28750


In [11]:
# https://www.tensorflow.org/tutorials/load_data/text#prepare_the_dataset_for_training
def int_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text), label

def tokenize(text):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text)

In [12]:
# Retrieve a batch (of 32 abstracts and titles) from the dataset.
text_batch, label_batch = next(iter(train_dataset.batch(32)))
first_abstract, first_label = text_batch[0], label_batch[0]


In [13]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [16]:
path_to_file = 'train.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

vocab = sorted(set(text))

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [17]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [19]:
dataset = train_dataset

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


UnimplementedError: ignored